In [1]:
%load_ext watermark
%watermark -a "Sushmit Roy" -u -d -p numpy,pandas,matplotlib,sklearn

Sushmit Roy 
last updated: 2018-01-10 

numpy 1.13.3
pandas 0.20.3
matplotlib 2.1.0
sklearn 0.19.1


In [2]:
## plotly libraries
import plotly
print(plotly.__version__)
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace


2.2.3


In [3]:
from sklearn import datasets
import numpy as np
import pandas as pd
import scipy as sc

In [4]:
iris = datasets.load_iris()
X = iris.data[:,[2,3]]
y = iris.target
print('Class labels:', np.unique(y))

Class labels: [0 1 2]


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =1 , stratify =y)

In [6]:
print('Labels counts in y:', np.bincount(y))
print('Labels counts in y test:', np.bincount(y_test))
print('Labels counts in y train:', np.bincount(y_train))

Labels counts in y: [50 50 50]
Labels counts in y test: [15 15 15]
Labels counts in y train: [35 35 35]


In [7]:
from sklearn.preprocessing import  StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [8]:
from sklearn.linear_model import Perceptron
ppn = Perceptron(max_iter=40, eta0=0.1, random_state=1)
ppn.fit(X_train_std, y_train)

Perceptron(alpha=0.0001, class_weight=None, eta0=0.1, fit_intercept=True,
      max_iter=40, n_iter=None, n_jobs=1, penalty=None, random_state=1,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [9]:
y_pred = ppn.predict(X_test_std)
print('Misclassified Samples: %d'% (y_test != y_pred).sum())

Misclassified Samples: 3


In [10]:
from sklearn.metrics import  accuracy_score
print('Accuracy: %.2f' %accuracy_score(y_test,y_pred))

Accuracy: 0.93


In [11]:
print('Accuracy: %.2f' % ppn.score(X_test_std,y_test))

Accuracy: 0.93


In [12]:
def plot_decision_regions_plotly(X, y, classifier, test_idx=None,resolution=0.02):
    # plot the decision surface
    data = []
    markers = ['square-open', 'x', 'o', '^', 'v']
    colors = ['red', 'blue', 'green', 'gray', 'cyan']
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)

    tx=np.arange(x1_min, x1_max,resolution)
    ty = np.arange(x2_min, x2_max, resolution)
    trace= go.Contour(
            z=Z,
            x=tx,
            y=ty,
    showscale = False,
    colorscale= [[value,colors[index] ]for index,value in enumerate(np.linspace(0.0, 1.0, len(np.unique(y))))],
        opacity = 0.4
        )
    data.append(trace)
    # plot class samples
    for idx, cl in enumerate(np.unique(y)):
        # Create a trace
        trace = go.Scatter(
            x =X[y == cl, 0],
            y =X[y == cl, 1],
            mode = 'markers',
            marker = dict(symbol =markers[idx],
                         color=colors[idx]  ),
            name = cl
        )
        data.append(trace)
        

    layout = go.Layout(
        width=600,
        height=400,
        xaxis= dict(
            zeroline= True,
            showline=True
        ),
        yaxis= dict(
            showline=True
        )
    )
    # highlight test samples
    if test_idx:
        # plot all samples
        X_test, y_test = X[test_idx,: ], y[test_idx]
        trace = go.Scatter(
        x= X_test[:,0],
        y= X_test[:,1],
        mode = 'markers',
        marker = dict(symbol = 100 ,
                     color = 'black',
                     size =8),
        name = 'test set'
        )
        data.append(trace)

    fig = go.Figure(data=data,layout=layout)
    fig['layout']['xaxis'].update(showgrid= False)
    fig['layout']['yaxis'].update(showgrid= False)
    fig['layout']['yaxis'].update(zeroline=False)
    fig['layout']['xaxis'].update(zeroline=False)
    fig['layout']['legend'].update(x=0,y=1)
    return fig

In [13]:
X_combined_std = np.vstack((X_train_std,X_test_std))
y_combined = np.hstack((y_train,y_test))
fig=plot_decision_regions_plotly(X=X_combined_std,y=y_combined,classifier=ppn,test_idx=range(105,150))
fig['layout']['xaxis'].update(title = 'petal length[standardized]')
fig['layout']['yaxis'].update(title = 'petal width [standardized]')
iplot(fig)

In [14]:
z = np.arange(-7,7,0.1)
def sigmoid(z):
    return  1.0/(1 + np.exp(-z))
phi_z = sigmoid(z)
trace = go.Scatter(
    x = z,
    y = phi_z,
    mode = 'line',

)

layout = dict(
              xaxis = dict(title = 'z'),
              yaxis = dict(title = 'phi_z'),
              )
data = [trace]
fig = dict(data=data, layout=layout)

iplot(fig)

In [15]:
def cost_1(z):
    return - np.log(sigmoid(z))
def cost_0(z):
    return - np.log(1 - sigmoid(z))
z = np.arange(-10,10,0.1)
phi_z = sigmoid(z)
c1 = [cost_1(x) for x in z]
c0 = [cost_0(x) for x in z]



trace1 = go.Scatter(
    x = phi_z,
    y = c0,
    name = 'J(w) if y=0',
    line = dict(
        width = 1,
        dash = 'dash')

)

trace2 = go.Scatter(
    x = phi_z,
    y = c1,
    name = 'J(w) if y=1',
    mode = 'line',

)

layout = dict(width=600,
              height=400,
              xaxis = dict(title = 'phi_z',range=[0, 1]),
              yaxis = dict(title = 'J(w)', range=[0, 5.1]),
              legend=dict(x=0.4, y=0.95)
              )
data = [trace1,trace2]

fig = dict(data=data, layout=layout)

iplot(fig)

In [16]:
class LogisticRegressionGD(object):
    """Logistic Regression Classifier using gradient descent.

    Parameters
    ------------
    eta : float
      Learning rate (between 0.0 and 1.0)
    n_iter : int
      Passes over the training dataset.
    random_state : int
      Random number generator seed for random weight
      initialization.


    Attributes
    -----------
    w_ : 1d-array
      Weights after fitting.
    cost_ : list
      Sum-of-squares cost function value in each epoch.

    """
    def __init__(self, eta=0.05, n_iter=100, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """ Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
          Training vectors, where n_samples is the number of samples and
          n_features is the number of features.
        y : array-like, shape = [n_samples]
          Target values.

        Returns
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.cost_ = []

        for i in range(self.n_iter):
            net_input = self.net_input(X)
            output = self.activation(net_input)
            errors = (y - output)
            self.w_[1:] += self.eta * X.T.dot(errors)
            self.w_[0] += self.eta * errors.sum()
            
            # note that we compute the logistic `cost` now
            # instead of the sum of squared errors cost
            cost = -y.dot(np.log(output)) - ((1 - y).dot(np.log(1 - output)))
            self.cost_.append(cost)
        return self
    
    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def activation(self, z):
        """Compute logistic sigmoid activation"""
        return 1. / (1. + np.exp(-np.clip(z, -250, 250)))

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, 0)
        # equivalent to:
        # return np.where(self.activation(self.net_input(X)) >= 0.5, 1, 0)

In [17]:
X_train_01_subset = X_train[(y_train ==0) | (y_train ==1)]
y_train_01_subset = y_train[(y_train ==0) | (y_train ==1)]
lrgd = LogisticRegressionGD(eta = 0.05, n_iter=1000, random_state=1)
lrgd.fit(X_train_01_subset, y_train_01_subset)
fig=plot_decision_regions_plotly(X=X_train_01_subset, y= y_train_01_subset, classifier=lrgd)
fig['layout']['xaxis'].update(title = 'petal length[standardized]')
fig['layout']['yaxis'].update(title = 'petal width [standardized]')
iplot(fig)

In [27]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=100.0,random_state=1)
lr.fit(X_train_std,y_train)

fig=plot_decision_regions_plotly(X_combined_std, y_combined, classifier=lr,test_idx=range(105,150))
fig['layout']['xaxis'].update(title = 'petal length[standardized]')
fig['layout']['yaxis'].update(title = 'petal width [standardized]')
iplot(fig)

In [28]:
lr.predict_proba(X_test_std[:3,:])

array([[  3.20136878e-08,   1.46953648e-01,   8.53046320e-01],
       [  8.34428069e-01,   1.65571931e-01,   4.57896429e-12],
       [  8.49182775e-01,   1.50817225e-01,   4.65678779e-13]])

In [31]:
lr.predict_proba(X_test_std[:3,:]).argmax(axis=1)

array([2, 0, 0], dtype=int64)

In [32]:
lr.predict(X_test_std[:3,:])

array([2, 0, 0])

In [18]:
np.unique(y_train_01_subset)

array([0, 1])

In [44]:
weights, params = [], []
for c in np.arange(-5,5):
    lr = LogisticRegression(C= 10.**c, random_state=1)
    lr.fit(X_train_std,y_train)

In [52]:
lr = LogisticRegression(C=100.0,random_state=1)
lr.fit(X_train_std,y_train)
test=lr.coef_

In [53]:
test.shape

(3, 2)